In [33]:
import xml.etree.ElementTree as ET
import numpy as np

import meshplot as mp

In [149]:
def load_control(control, args):
    if control is None:
        return
        
    tsn = control.find("time_steps")
    ssn = control.find("step_size")
    an = control.find("analysis")

    
    if tsn is not None and ssn is not None and an is not None:
        if an.attrib["type"] == "dynamic":
            time_steps = int(tsn.text)
            step_size = float(ssn.text)

            args["time"] = {
                "tend": step_size * time_steps,
                "time_steps": time_steps
            }

In [156]:
def load_materials(febio):
    materials = {}

    material_parent = febio.find("Material")
    
    for material_node in material_parent.iter("material"):
        material = material_node.attrib["type"]
        mid = int(material_node.attrib["id"])
        
        E = float(material_node.find("E").text)
        nu = float(material_node.find("v").text)
                  
        if material_node.find("density") is None:
            rho = 1
        else:
            rho = float(material_node.find("density").text)

        mat = ""
        if material == "neo-Hookean":
            mat = "NeoHookean"
        elif material == "isotropic elastic":
            mat = "LinearElasticity"
        else:
            print("Unsupported material {}, reverting to isotropic elastic".format(material))
            mat = "LinearElasticity"
            

        materials[mid] = (E, nu, rho, mat)
        
        
        return materials

In [150]:
def load_nodes(geometry):
    vertices = []
    for nodes in geometry.iter("Nodes"):
        for child in nodes.iter("node"):
            pos_str = child.text
            vs = pos_str.split(",")
            assert(len(vs) == 3);
            
            vertices.append([float(vs[0]), float(vs[1]), float(vs[2])])


    V = np.array(vertices)
    
    return V

In [151]:
def load_elements(geometry, numV, materials):
    els = []
    nodes = []
    mids = []
    
    order = 1
    
    is_hex = False
    types = ""
    
    for elements in geometry.iter("Elements"):
        el_type = elements.attrib["type"]
        mid = int(elements.attrib["mat"])

        if el_type != "tet4" and el_type != "tet10" and el_type != "tet20" and el_type != "hex8":
            print("Unsupported elemet type {}".format(el_type))
            continue


        if len(types) == 0:
            if el_type.startswith("tet"):
                types = "tet"
            else:
                types = "hex"
        elif not el_type.startswith(types, 0):
            print("Unsupported elemet type {} since the mesh contains also {}".format(el_type, types))
            continue

        if el_type == "tet4":
            order = max(1, order)
        elif el_type == "tet10":
            order = max(2, order)
        elif el_type == "tet20":
            order = max(3, order)
        elif el_type == "hex8":
            order = max(1, order)
            is_hex = True
                

        for child in elements.iter("elem"):
                ids = child.text
                tt = ids.split(",");
                assert(len(tt) >= 4);
                
                node_size = 8 if is_hex else 4

                els.append([])
                
                for n in range(node_size):
                    els[-1].append(int(tt[n]) - 1)
                    assert(els[-1][n] < numV)

                nodes.append([])
                mids.append(mid)
                
                for n in range(len(tt)):
                    nodes[-1].append(int(tt[n]) - 1)

                if el_type == "tet10":
                    assert(len(nodes[-1]) == 10)
                    nodes[-1][8], nodes[-1][9] = nodes[-1][9], nodes[-1][8]
                elif el_type == "tet20":
                    assert(len(nodes[-1]) == 20)
                    nodes[-1][8], nodes[-1][9] = nodes[-1][9], nodes[-1][8]
                    nodes[-1][10], nodes[-1][11] = nodes[-1][11], nodes[-1][10]
                    nodes[-1][12], nodes[-1][15] = nodes[-1][15], nodes[-1][12]
                    nodes[-1][13], nodes[-1][14] = nodes[-1][14], nodes[-1][13]
                    nodes[-1][16], nodes[-1][19] = nodes[-1][19], nodes[-1][16]
                    nodes[-1][17], nodes[-1][19] = nodes[-1][19], nodes[-1][17]


    T = np.array(els)

# 				T.resize(els.size(), is_hex ? 8 : 4);
# 				Es.resize(els.size(), 1);
# 				nus.resize(els.size(), 1);
# 				rhos.resize(els.size(), 1);
# 				mats.resize(els.size());
# 				for (int i = 0; i < els.size(); ++i)
# 				{
# 					T.row(i) = els[i].transpose();
# 					const auto it = materials.find(mids[i]);
# 					assert(it != materials.end());
# 					if (it == materials.end())
# 					{
# 						logger().error("Unable to find material {}", mids[i]);
# 						throw std::runtime_error("Invalid material");
# 					}
# 					Es(i) = std::get<0>(it->second);
# 					nus(i) = std::get<1>(it->second);
# 					rhos(i) = std::get<2>(it->second);
# 					mats[i] = std::get<3>(it->second);
# 				}

# 				return order;
# 			}
                
    return T

In [152]:
output_json = {}
output_json["output"] = {
    "json": "sim.json",
    "paraview": {
            "file_name": "sim.pvd",
            "surface": True,
            "options": {
                "material": True,
                "body_ids": True
            },
            "vismesh_rel_area": 10000000
        }
}

In [153]:
# tree = ET.parse("../data/test.feb")
tree = ET.parse("../data/lin-neo.feb")

root = tree.getroot()

if root.attrib["version"] != "2.5":
    assert(False)
    
control = root.find("Control")
load_control(control, output_json)
    
    
geometry = root.find("Geometry");
has_collisions = geometry.find("SurfacePair")

V = load_nodes(geometry)

T = load_elements(geometry, V.shape[0], {})

output_json

{'output': {'json': 'sim.json',
  'paraview': {'file_name': 'sim.pvd',
   'surface': True,
   'options': {'material': True, 'body_ids': True},
   'vismesh_rel_area': 10000000}},
 'time': {'tend': 1.0, 'time_steps': 10}}

In [139]:
mp.plot(V, T, shading={"point_size": 1, "wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [26]:
asd

<Element 'Globals' at 0x7fd3f11eb098>

In [27]:
root.find("Globals")

<Element 'Globals' at 0x7fd3f11eb098>